# Notebook - 4 - SQL Server

### Topics Covered in this notebook

1. Transactions in SQL Server
2. Transactions ACID Test in SQL Server
3. Subqueries in SQL Server
4. Correlated Subquery in SQL Server

In [1]:
USE kudvenkatdb_master;

Commands completed successfully.

Total execution time: 00:00:00.016

In [5]:
SELECT * FROM INFORMATION_SCHEMA.TABLES

(27 rows affected)

Total execution time: 00:00:00.065

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
KudvenkatDb_Master,dbo,vwEmployeesByDepartment,VIEW
KudvenkatDb_Master,dbo,vwSummarizedData,VIEW
KudvenkatDb_Master,dbo,vwEmployeesDataExceptSalary,VIEW
KudvenkatDb_Master,dbo,tblProduct,BASE TABLE
KudvenkatDb_Master,dbo,tblProductSales,BASE TABLE
KudvenkatDb_Master,dbo,vwTotalSalesByproduct,VIEW
KudvenkatDb_Master,dbo,tblAudit,BASE TABLE
KudvenkatDb_Master,dbo,tblEmployeeManager,BASE TABLE
KudvenkatDb_Master,dbo,tblProductSales2,BASE TABLE
KudvenkatDb_Master,dbo,tblProductSales3,BASE TABLE


## Transactions
### What is a Transaction?
A transaction is a group of commands that change the data stored in a database. A transaction, is treated as a single unit. A transaction ensures that, either all of the commands succeed, or none of them. If one of the commands in the transaction fails, all of the commands fail, and any data that was modified in the database is rolled back. In this way, transactions maintain the integrity of data in a database.

**Transaction processing follows these steps:**
1. Begin a transaction.
2. Process database commands.
3. Check for errors.

        if errors occured,
            rollback the transaction,
        else
            commit the transaction    
**Note**: NOT able to see the un-committed changes

***SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED***

In [2]:
CREATE TABLE tblPhysicalAddress(
    AddressId int PRIMARY key,
    EmployeeNumber int,
    HouseNumber NVARCHAR(30),
    StreetAddress NVARCHAR(30),
    City NVARCHAR(10)
);

CREATE TABLE tblMailingAddress(
    AddressId int PRIMARY key,
    EmployeeNumber int,
    HouseNumber NVARCHAR(30),
    StreetAddress NVARCHAR(30),
    City NVARCHAR(10)
);

Commands completed successfully.

Total execution time: 00:00:00.023

In [ ]:
INSERT INTO tblMailingAddress VALUES(1,101,'#10','King Street','LONDOON')
INSERT INTO tblPhysicalAddress VALUES(1,101,'#10','King Street','LONDOON')

In [6]:
SELECT * FROM tblPhysicalAddress;
SELECT * FROM tblMailingAddress;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.043

AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,LONDOON


AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,LONDOON


In [7]:
-- create a stored procedure for updating the City Column

CREATE PROCEDURE spUpdateAddress
as
BEGIN
    BEGIN TRY
        BEGIN TRANSACTION
            UPDATE tblPhysicalAddress SET City = 'London' WHERE AddressId = 1 -- first updating statement

            UPDATE tblMailingAddress SET City = 'London' WHERE AddressId = 1 -- 2nd updating statement
        COMMIT TRANSACTION
        PRINT 'Transaction Committed'
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION
        PRINT 'Transaction Rolled Back'
    END CATCH
END


Commands completed successfully.

Total execution time: 00:00:00.006

In [8]:
EXEC spUpdateAddress

(1 row affected)

(1 row affected)

Transaction Committed

Total execution time: 00:00:00.010

In [9]:
SELECT * FROM tblPhysicalAddress;
SELECT * FROM tblMailingAddress;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.020

AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,London


AddressId,EmployeeNumber,HouseNumber,StreetAddress,City
1,101,#10,King Street,London


In [10]:
-- here we intentionally added the error to watch the status as  'Transaction Rolled Back'
ALTER PROCEDURE spUpdateAddress
as
BEGIN
    BEGIN TRY
        BEGIN TRANSACTION
            UPDATE tblPhysicalAddress SET City = 'London' WHERE AddressId = 1 -- first updating statement

            UPDATE tblMailingAddress SET City = 'London LONDON' WHERE AddressId = 1 -- 2nd updating statement
        COMMIT TRANSACTION
        PRINT 'Transaction Committed'
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION
        PRINT 'Transaction Rolled Back'
    END CATCH
END

Commands completed successfully.

Total execution time: 00:00:00.026

In [11]:
EXEC spUpdateAddress

(1 row affected)

(0 rows affected)

Transaction Rolled Back

Total execution time: 00:00:00.009

## Transaction ACID Test

**A transaction is a group of database commands that are treated as a single unit.** A successful transaction must pass the "ACID" test, that is, it must be


**Atomic**-All statements in the transaction either completed successfully or they were all rolled back. The task that the set of operations represents is either accomplished or not, but in any case not left half-done.


**Consistent** - All data touched by the transaction is left in a logically consistent state. For example, if stock available numbers are decremented from tblProductTable, then, there has to be a related entry in tblProductSales table. The inventory can't just disappear.

**Isolated:** The transaction must affect data without interfering with other concurrent transactions, or being interfered with by them. This prevents transactions from making changes to data based on uncommitted information, for example changes to a record that are subsequently rolled back. Most databases use locking to maintain transaction isolation.


**Durable:** Once a change is made, it is permanent. If a system error or power failure occurs before a set of commands is complete, those commands are undone and the data is restored to its original state once the system begins running again.

## Subqueries
     A subquery is simply a select statement, that returns a single value and can be nested inside a SELECT, UPDATE, INSERT, or DELETE statement. It is also possible to nest a subquery inside another subquery. According to MSDN, subqueries can be nested upto 32 levels.

     Subqueries are always encolsed in paranthesis and are also called as inner queries, and the query containing the subquery is called as outer query. The columns from a table that is present only inside a subquery, cannot be used in the SELECT list of the outer query.

In [2]:
Create Table tblProductsSQ
(
 [Id] int identity primary key,
 [Name] nvarchar(50),
 [Description] nvarchar(250)
)

Create Table tblProductSalesSQ
(
 Id int primary key identity,
 ProductId int foreign key references tblProductsSQ(Id),
 UnitPrice int,
 QuantitySold int
)

Commands completed successfully.

Total execution time: 00:00:00.004

In [ ]:
Insert into tblProductsSQ values ('TV', '52 inch black color LCD TV')
Insert into tblProductsSQ values ('Laptop', 'Very thin black color acer laptop')
Insert into tblProductsSQ values ('Desktop', 'HP high performance desktop')

Insert into tblProductSalesSQ values(3, 450, 5)
Insert into tblProductSalesSQ values(2, 250, 7)
Insert into tblProductSalesSQ values(3, 450, 4)
Insert into tblProductSalesSQ values(3, 450, 9)

In [4]:
-- Write a query to retrieve products that are not at all sold?
SELECT Id, [Name], [Description]
FROM	tblProductsSQ
WHERE Id NOT IN (SELECT ProductId FROM tblProductSalesSQ)


-- Most of the times subqueries can be very easily replaced with joins. 
-- The above query is rewritten using joins and produces the same results. 

SELECT tP.Id, tP.Name, tP.Description
FROM tblProductsSQ AS tP
LEFT JOIN tblProductSalesSQ as tS
ON tS.ProductId = tP.Id
WHERE tS.ProductId IS NULL

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.069

Id,Name,Description
1,TV,52 inch black color LCD TV


Id,Name,Description
1,TV,52 inch black color LCD TV


In [5]:
-- Let us now discuss about using a sub query in the SELECT clause. 
-- Write a query to retrieve the NAME and TOTALQUANTITY sold, using a subquery.

SELECT [Name], (SELECT SUM(QuantitySold) from tblProductSalesSQ WHERE ProductId = tblProductsSQ.Id) as TotalQuntySold
FROM tblProductsSQ ORDER BY NAME

-- Query with an equivalent join that produces the same result.

SELECT [Name], SUM(tS.QuantitySold) as TotalQntySold
from tblProductsSQ as tP
Left Join tblProductSalesSQ as tS
on tP.Id = tS.ProductId
group by tP.Name

(3 rows affected)

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.015

Name,TotalQuntySold
Desktop,18
Laptop,7
TV,NULL


Name,TotalQntySold
Desktop,18
Laptop,7
TV,NULL


## Correlated Subquery
**If the subquery depends on the outer query for its values,** then that sub query is called as a correlated subquery.

**In the where clause of the subquery below,** "ProductId" column get it's value from tblProductsSQ table that is present in the outer query.
    
    SELECT [Name], (SELECT SUM(QuantitySold) from tblProductSalesSQ WHERE ProductId = tblProductsSQ.Id) as TotalQuntySold FROM tblProductsSQ ORDER BY NAME

**So, here the subquery is dependent on the outer query for it's value,** hence this subquery is a correlated subquery. 

**Correlated subqueries get executed,** once for every row that is selected by the outer query. 

**Corelated subquery**, cannot be executed independently of the outer query.

## Inserting Large amount of data for performance testing

In [9]:
DROP VIEW vwTotalSalesByproduct

Commands completed successfully.

Total execution time: 00:00:00.023

In [12]:
IF (EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'tblProductSales'))
BEGIN
    drop TABLE tblProductSales
END

IF (EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'tblProducts'))
BEGIN
    drop TABLE tblProducts
END

Commands completed successfully.

Total execution time: 00:00:00.006

In [13]:
CREATE TABLE tblProducts(
    Id int IDENTITY PRIMARY KEY,
    Name NVARCHAR(50),
    DESCRIPTION NVARCHAR(250)
);

CREATE TABLE tblProductSales(
    Id int PRIMARY KEY IDENTITY,
    ProductId int FOREIGN KEY REFERENCES tblProducts(Id),
    UnitPrice INT,
    QuantitySold INT
);

Commands completed successfully.

Total execution time: 00:00:00.015

Insert Sample data into tblProducts table

In [4]:
Declare @Id INT
Set @Id = 1

WHILE(@Id <= 50)
BEGIN
    Insert into tblProducts values ('Product - ' + CAST(@Id as nvarchar(20)),
                                    'Product - ' + CAST(@Id as nvarchar(20)) + ' Description')
    --PRINT @Id
    Set @Id = @Id + 1
END

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.016

In [5]:
--DELETE from tblProducts
SELECT * FROM tblProducts

(50 rows affected)

Total execution time: 00:00:00.015

Id,Name,DESCRIPTION
10051,Product - 1,Product - 1 Description
10052,Product - 2,Product - 2 Description
10053,Product - 3,Product - 3 Description
10054,Product - 4,Product - 4 Description
10055,Product - 5,Product - 5 Description
10056,Product - 6,Product - 6 Description
10057,Product - 7,Product - 7 Description
10058,Product - 8,Product - 8 Description
10059,Product - 9,Product - 9 Description
10060,Product - 10,Product - 10 Description


In [27]:
DECLARE @LL INT
SET @LL = 1

DECLARE @UL INT
SET @UL = 5

SELECT ROUND((@UL - @LL) * RAND() + 1, 0)

(1 row affected)

Total execution time: 00:00:00.004

(No column name)
1


In [6]:
-- generate random no's b/w 1-5

DECLARE @LL INT
SET @LL = 1

DECLARE @UL INT
SET @UL = 5

DECLARE @Rand int
while ( 1= 1)
BEGIN
    SELECT @Rand = ROUND((@UL - @LL) * RAND() + 1, 0)
    Print @Rand

    if (@Rand < 1 OR @Rand >= 5)
    BEGIN   
        PRINT 'Error - ' + CAST(@Rand as NVARCHAR(4))
        BREAK
    END
END

3

5

Error - 5

Total execution time: 00:00:00.001

## Performance - SubQueries or Joins

**According to MSDN, in most cases,** there is usually no performance difference between queries that uses sub-queries and equivalent queries using joins.

**According to MSDN, in some cases where existence must be checked,** a join produces better performance. Otherwise, the nested query must be processed for each result of the outer query. In such cases, a join approach would yield better results.

**In general joins work faster than sub-queries,** but in reality it all depends on the execution plan that is generated by SQL Server. Id does not matter how we have written the query, SQL Server will always transform it on an execution plan. If it is "smart" enough to generate the same plan from both queries, you will get the same result.

**I would say, rather than going by theory, turn on client statistics and execution plan to see the performance of each option, and then make a decision.** In a later video session we will discuss about client statistics and execution plans in detail.